In [1]:
!pip install thefuzz
!pip install nltk
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_lg-0.5.3.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_lg-0.5.3.tar.gz (531.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import nltk
from load_data import load_ann, load_txt
import pandas as pd
from thefuzz import fuzz
from nltk.stem import WordNetLemmatizer
import spacy
from scispacy.abbreviation import AbbreviationDetector

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

abreviation_handler = spacy.load("en_core_sci_lg")
abreviation_handler.add_pipe("abbreviation_detector")


abreviation_handler = spacy.load("en_core_sci_lg")
abreviation_handler.add_pipe("abbreviation_detector")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/codespace/.python/current/lib/python3.10/site-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [3]:
PATH_TO_ZIP = "/workspaces/codespaces-jupyter/Project/RawData"
DATA_PATH = f"{PATH_TO_ZIP}/"
print(f"Full data path: {DATA_PATH}")
# read in txt files
txt_df = load_txt(DATA_PATH)
# read in REASONS entities from .ann files
ent_df, rel_df = load_ann(DATA_PATH)

Full data path: /workspaces/codespaces-jupyter/Project/RawData/
Time taken to read .txt files: 0.009160757064819336
Time taken to read .ann files and extract all metadata: 0.12025904655456543


# Basic EDA

In [4]:
# EDA on txt_df
print("txt_df shape:", txt_df.shape)
print("txt_df columns:", txt_df.columns)
print("txt_df head:\n", txt_df.head())

# EDA on ent_df
print("ent_df shape:", ent_df.shape)
print("ent_df columns:", ent_df.columns)
print("ent_df head:\n", ent_df.head())

# EDA on rel_df
print("rel_df shape:", rel_df.shape)
print("rel_df columns:", rel_df.columns)
print("rel_df head:\n", rel_df.head())

txt_df shape: (303, 2)
txt_df columns: Index(['file_idx', 'text'], dtype='object')
txt_df head:
   file_idx                                               text
0   109450  Admission Date:  [**2121-8-7**]              D...
1   103677  Admission Date:  [**2128-12-3**]              ...
2   113824  Admission Date:  [**2200-6-14**]              ...
3   113524  Admission Date:  [**2124-1-14**]              ...
4   115244  Admission Date:  [**2168-4-18**]     Discharge...
ent_df shape: (50951, 6)
ent_df columns: Index(['file_idx', 'entity_id', 'category', 'start_idx', 'end_idx', 'text'], dtype='object')
ent_df head:
   file_idx entity_id  category start_idx end_idx             text
0   120253        T1      Drug     10002   10015  Calcipotriene\n
1   120253        T2  Strength     10016   10021         0.005 \n
2   120253        T3      Form     10024   10029          Cream\n
3   120253        T4    Dosage     10035   10042        One (1)\n
4   120253        T5      Form     10043   10047     

# Data Cleaning

In [5]:
"""
1. Convert 'text' columns to lowercase, in order to facilitate comparison.
"""
# To lowercase 'text' column in ent_df
ent_df['text'] = ent_df['text'].str.lower()
ent_df['text'] = ent_df['text'].str.strip()

# To lowercase 'text' column in txt_df
# txt_df['text'] = txt_df['text'].str.lower()

"""
2. Remove \n ending from 'text' column in ent_df and in 'entity2' column in rel_df.
"""
ent_df['text'] = ent_df['text'].str.rstrip('\n')
rel_df['entity2'] = rel_df['entity2'].str.rstrip('\n')

"""
3. Convert 'start_idx' and 'end_idx' columns in ent_df to int.
"""
# Drop rows that cannot be converted to int TODO: Make this better
ent_df = ent_df[ent_df['start_idx'].str.isnumeric()]
ent_df = ent_df[ent_df['end_idx'].str.isnumeric()]
ent_df['start_idx'] = ent_df['start_idx'].astype(int)
ent_df['end_idx'] = ent_df['end_idx'].astype(int)

# Make new column with lemmatized text of 'text' column called 'lemmatized_text'
ent_df['orig_txt'] = ent_df['text']
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

ent_df['text'] = ent_df['text'].apply(lemmatize_text)
ent_df['text'] = ent_df['text'].astype(str)

# Feature Engineering

In [6]:
"""
1. Join the appropriate entity1 and entity2 for each relation in rel_df.
"""
# Remove first 5 letters from 'entity1' and 'entity2' column in rel_df
rel_df['entity1'] = rel_df['entity1'].str[5:]
rel_df['entity2'] = rel_df['entity2'].str[5:]
rel_df = rel_df.merge(ent_df[['entity_id', 'text', 'file_idx']], how='left', left_on=['entity1', 'file_idx'], right_on=['entity_id', 'file_idx'])
rel_df.rename(columns={'text': 'entity1_text'}, inplace=True)
rel_df = rel_df.merge(ent_df[['entity_id', 'text', 'file_idx']], how='left', left_on=['entity2', 'file_idx'], right_on=['entity_id', 'file_idx'])
rel_df.rename(columns={'text': 'entity2_text'}, inplace=True)
rel_df.drop(columns=['entity_id_x', 'entity_id_y'], inplace=True)

# Create column 'entity1_entity2' in rel_df
rel_df['entity1_entity2'] = rel_df['entity1_text'] + rel_df['entity2_text']

"""
2. Get count of text in file_idx for each entity in ent_df. Do the same for the 'entity1_entity2' in rel_df.
"""
ent_df_count = ent_df.groupby(['text', 'file_idx']).size().reset_index(name='count_in_document')
ent_df = ent_df.merge(ent_df_count, how='left', left_on=['text', 'file_idx'], right_on=['text', 'file_idx'])
rel_df_count = rel_df.groupby(['entity1_entity2', 'file_idx']).size().reset_index(name='count_in_document')
rel_df = rel_df.merge(rel_df_count, how='left', left_on=['entity1_entity2', 'file_idx'], right_on=['entity1_entity2', 'file_idx'])

"""
3. Create encoding to represent if entity in ent_df is in the 'Discharge Diagnosis', 'Chief Complaint', or 'History of Present Illness' section of the txt_df.
"""
def find_section_range(row, section_name):
    lines = row['text'].split('\n')
    matches = [(i, fuzz.ratio(line.lower(), section_name.lower())) for i, line in enumerate(lines)]
    matches.sort(key=lambda x: x[1], reverse=True)  # sort by fuzz.ratio in descending order
    if not matches:
        # Raise error if no match is found
        raise ValueError(f"Could not find section {section_name} in file {row['file_idx']}")
    start_line = matches[0][0]  # start of the range is the line with the highest fuzz.ratio
    end_line = start_line
    while end_line < len(lines) and lines[end_line].strip() != '':
        end_line += 1
    # calculate start and end index within the raw text
    start_index = sum(len(line) + 1 for line in lines[:start_line])  # +1 for the newline character
    end_index = sum(len(line) + 1 for line in lines[:end_line])  # +1 for the newline character
    return (start_index, end_index)

txt_df['DD_Range'] = txt_df.apply(lambda row: find_section_range(row, 'Discharge Diagnosis'), axis=1)
txt_df['CC_Range'] = txt_df.apply(lambda row: find_section_range(row, 'Chief Complaint'), axis=1)
txt_df['HPI_Range'] = txt_df.apply(lambda row: find_section_range(row, 'History of Present Illness'), axis=1)

# Join the 'DD_Range', 'CC_Range', and 'HPI_Range' columns from txt_df to ent_df
ent_df = ent_df.merge(txt_df[['file_idx', 'DD_Range', 'CC_Range', 'HPI_Range']], how='left', left_on=['file_idx'], right_on=['file_idx'])

# Loop through each entity in ent_df and check if it is in the 'Discharge Diagnosis', 'Chief Complaint', or 'History of Present Illness' section of the txt_df.
# If it is, then add the section name to the 'section' column in ent_df.
def find_section(row):
    # Throw error if start_idx is greater than end_idx
    if row['start_idx'] > row['end_idx']:
        raise ValueError(f"start_idx {row['start_idx']} is greater than end_idx {row['end_idx']}")
    
    # If start_idx and end_idx are in one section, return the section name
    if row['start_idx'] >= row['DD_Range'][0] and row['end_idx'] <= row['DD_Range'][1]:
        return 'Discharge Diagnosis'
    elif row['start_idx'] >= row['CC_Range'][0] and row['end_idx'] <= row['CC_Range'][1]:
        return 'Chief Complaint'
    elif row['start_idx'] >= row['HPI_Range'][0] and row['end_idx'] <= row['HPI_Range'][1]:
        return 'History of Present Illness'
    else:
        return 'Other'

ent_df['section'] = ent_df.apply(lambda row: find_section(row), axis=1)
# Drop DD_Range, CC_Range, and HPI_Range columns from ent_df
ent_df.drop(columns=['DD_Range', 'CC_Range', 'HPI_Range'], inplace=True)
# Apply one hot encoding to 'section' column in ent_df
ent_df = pd.get_dummies(ent_df, columns=['section'])

In [7]:
"""
# QUESTION: What are the categories of entities and relationships? Do they encapsulate primary medical diagnoses and common unerlying factors?"
"""
# Store unique values in 'category' in ent_df and the count of each, and an example of the category in a DF
ent_df_unique = pd.DataFrame(ent_df['category'].value_counts())
# Get an example of each category and add it as a column to ent_df_unique
ent_df_unique['example'] = ent_df.groupby('category')['text'].apply(lambda x: x.sample(1).values[0])
print(ent_df_unique.head(30))


# Store unique values in 'category' in rel_df and the count of each in a DF
rel_df_unique = pd.DataFrame(rel_df['category'].value_counts())
print(rel_df_unique.head(30))

"""
# LEARNING: 
# 1. The categories of entities and relationships are not mutually exclusive. For example, a patient can have both a primary diagnosis and a secondary diagnosis.
# 2. All relationships tie entities to drugs
"""

"""
# FURTHER QUESTIONS TO EXPLORE:
# 1. Which category types best represent the medical diagnosis and the common underlying factors in ent_df and rel_df?
# 2. Can we derive the primary diagnosis by choosing the most common Reason?
# 3. Can we derive the primary diagnosis by choosing the Reason that is most closely related to most frequently occuring drug?
# 4. Can we derive the primary diagnosis / underlying factors by choosing the Reason that occurs the most in the 'Discharge Diagnosis', 'Chief Complaint', or 'Hisotry of Present Illness' sections?
"""

           category           example
Drug          15192       propranolol
Strength       6465            180 mg
Form           6321           capsule
Route          5449       perm-a-cath
Frequency      4771             daily
Dosage         3926         one ( 1 )
Reason         3575  sattelite lesion
ADE             892  unresponsiveness
Duration        538          two week
                category
Strength-Drug       6702
Form-Drug           6654
Frequency-Drug      6310
Route-Drug          5538
Reason-Drug         5169
Dosage-Drug         4225
ADE-Drug            1107
Duration-Drug        643


"\n# FURTHER QUESTIONS TO EXPLORE:\n# 1. Which category types best represent the medical diagnosis and the common underlying factors in ent_df and rel_df?\n# 2. Can we derive the primary diagnosis by choosing the most common Reason?\n# 3. Can we derive the primary diagnosis by choosing the Reason that is most closely related to most frequently occuring drug?\n# 4. Can we derive the primary diagnosis / underlying factors by choosing the Reason that occurs the most in the 'Discharge Diagnosis', 'Chief Complaint', or 'Hisotry of Present Illness' sections?\n"

In [8]:
"""
# QUESTION: Which category types best represent the medical diagnosis and the common underlying factors in ent_df and rel_df?
"""

# Create a list of the categories in ent_df_unique, print the top 10 most occuring values in the 'text' for each category and count the number of unique values
print("Entities:\n")
ent_df_unique_list = ent_df_unique.index.tolist()
for category in ent_df_unique_list:
    print(f"Category: {category}")
    print(ent_df[ent_df['category'] == category]['text'].value_counts().head(100))
    print(f"Number of unique values: {ent_df[ent_df['category'] == category]['text'].nunique()}")
    print("\n")
    # If the table category is 'Reason', save value counts and save to csv
    if category == 'Reason':
        ent_df[ent_df['category'] == category]['text'].value_counts().to_csv('reason_df.csv')

# Create a list of the categories in rel_df_unique, print the top 10 most occuring 'entity1_text' and 'entity2_text' combinations seperated by a '-' and count the number of unique values
print("Relationships:\n")
rel_df_unique_list = rel_df_unique.index.tolist()
for category in rel_df_unique_list:
    print(f"Category: {category}")
    print(rel_df[rel_df['category'] == category]['entity1_text'].str.cat(rel_df[rel_df['category'] == category]['entity2_text'], sep='-').value_counts().head(100))
    print(f"Number of unique values: {rel_df[rel_df['category'] == category]['entity1_text'].str.cat(rel_df[rel_df['category'] == category]['entity2_text'], sep='-').nunique()}")
    print("\n")
    if category == 'Reason-Drug':
        rel_df[rel_df['category'] == category]['entity1_entity2'].value_counts().to_csv('reason_drug_df.csv')

"""
# LEARNING:
# 1. 'Reason' category for entities, as Dataset Overview PDF suggests, seems to be the best category to represent the medical diagnosis based on its categories
# 2. 'Reason-Drug' category for relationships seems to be the best category to represent the common underlying factors based on its categories
"""

"""
# HYPOTHESIS:
# 1. The most common 'Reason' category for entities in a document in the 'Discharge Diagnosis' section is the primary diagnosis
# 2. The most prevalent underlying factors are the most common reasons from the most commonly occuring 'Reason-Drug' relationship. This is because the doctor likely is using drugs to treat the most common underlying factors.
"""

"""
# FURTHER QUESTIONS TO EXPLORE:
# 1. Can we derive the primary diagnosis by choosing the most common Reason? Is it more accurate to choose the most common Reason in the 'Discharge Diagnosis' section?
# 2. Can we derive the primary diagnosis by choosing the Reason that is most closely related to most frequently occuring drug?
"""

Entities:

Category: Drug
coumadin               370
vancomycin             265
aspirin                255
lasix                  248
antibiotic             222
                      ... 
ipratropium bromide     36
daptomycin              35
anticoagulation         35
metformin               34
vitamin k               34
Name: text, Length: 100, dtype: int64
Number of unique values: 1874


Category: Strength
20 mg     323
10 mg     284
100 mg    283
40 mg     256
5 mg      256
         ... 
1gm        12
1 gm       12
240 mg     11
1 g        11
0.1 mg     11
Name: text, Length: 100, dtype: int64
Number of unique values: 716


Category: Form
tablet                                3362
capsule                                529
solution                               192
tablet , delayed release ( e.c . )     148
tab                                    118
                                      ... 
infatabs                                 3
disk                                     3
tablet

"\n# FURTHER QUESTIONS TO EXPLORE:\n# 1. Can we derive the primary diagnosis by choosing the most common Reason? Is it more accurate to choose the most common Reason in the 'Discharge Diagnosis' section?\n# 2. Can we derive the primary diagnosis by choosing the Reason that is most closely related to most frequently occuring drug?\n"

In [9]:
"""
# QUESTION: Can we derive the primary diagnosis by choosing the most common Reason? Is it more accurate to choose the most common Reason in the 'Discharge Diagnosis' section?
"""

# Check that every document has a 'Discharge Diagnosis' section
print(txt_df['text'][txt_df['text'].str.contains('diagnosis:')].count() / txt_df.shape[0])
# Check that every document has a 'Chief Complaint' section
print(txt_df['text'][txt_df['text'].str.contains('complaint:')].count() / txt_df.shape[0])
# Check that every document has a 'History of Present Illness' section
print(txt_df['text'][txt_df['text'].str.contains('illness:')].count()/ txt_df.shape[0])

# TODO: Figure out how to fuzzy match discharge diagonosis

# Use file_idx 100035 as an example. Create a txt_df_subset with only file_idx 100035
txt_df_subset = txt_df[txt_df['file_idx'] == '100035']
ent_df_subset = ent_df[ent_df['file_idx'] == '100035']
rel_df_subset = rel_df[rel_df['file_idx'] == '100035']

# Create a list of the categories in ent_df_unique, print the top 10 most occuring values that have category 'Reason'
print("Entities:\n")
ent_df_unique_list = ent_df_unique.index.tolist()
for category in ent_df_unique_list:
    if category == "Reason":
        print(f"Category: {category}")
        print(ent_df_subset[ent_df_subset['category'] == category]['text'].value_counts().head(30))
        print(f"Number of unique values: {ent_df_subset[ent_df_subset['category'] == category]['text'].nunique()}")
        print("\n")


# Print the top 10 most occuring values that have category 'Reason', where section_Chief Complaint is 1 or section_History of Present Illness is 1 or section_Discharge Diagnosis
print("Entities:\n")
ent_df_unique_list = ent_df_unique.index.tolist()
for category in ent_df_unique_list:
    if category == 'Reason' or category == 'Drug':
        print(f"Category: {category}")
        print(ent_df_subset[(ent_df_subset['category'] == category) & ((ent_df_subset['section_Chief Complaint'] == 1) | (ent_df_subset['section_History of Present Illness'] == 1) | (ent_df_subset['section_Discharge Diagnosis'] == 1))]['text'].value_counts().head(30))
        print(f"Number of unique values: {ent_df_subset[(ent_df_subset['category'] == category) & ((ent_df_subset['section_Chief Complaint'] == 1) | (ent_df_subset['section_History of Present Illness'] == 1) | (ent_df_subset['section_Discharge Diagnosis'] == 1))]['text'].nunique()}")
        print("\n")
    # If category is Drug, get all reasons that are related to drugs from rel_df
    if category == 'Drug':
        drugs = rel_df_subset[rel_df_subset['category'] == 'Reason-Drug']['entity1_text'].unique().tolist()
        print(f"Category: {category} updated")
        print(ent_df_subset[(ent_df_subset['category'] == 'Reason') & (ent_df_subset['text'].isin(drugs))]['text'].value_counts().head(100))
        print(f"Number of unique values: {ent_df_subset[(ent_df_subset['category'] == category) & (ent_df_subset['text'].isin(drugs))]['text'].nunique()}")
        print("\n")




0.0231023102310231
0.0
0.0
Entities:

Category: Reason
pain                               4
asthma                             3
constipation                       2
seizure                            2
fever                              2
agitation                          2
longer than 5 minute               1
back pain                          1
sob                                1
sleep                              1
recurrent seizure                  1
cap                                1
hypertension                       1
low bps                            1
elevated cr 1.9                    1
diuresed                           1
ventilator associated pneumonia    1
wheezing                           1
aggitated                          1
Name: text, dtype: int64
Number of unique values: 19


Entities:

Category: Drug
epi             2
ctx             2
azithromycin    2
bicarb          2
epinephrine     2
vecuronium      1
solumedrol      1
atropine        1
magnesium       1

In [10]:
"""
# QUESTION: How many docuements contain a 'ground truth' primary diagnosis, where the string 'primary' exists in the 'Discharge Diagnosis' section?
"""
# Loop through every document in txt_df and check 'text' column to if 'primary' exists in the 'Discharge Diagnosis' section
primary_diagnosis = []
for index, row in txt_df.iterrows():
    if 'primary' in row['text'][row['DD_Range'][0]:row['DD_Range'][1]].lower():
        primary_diagnosis.append(row['file_idx'])
print(len(primary_diagnosis) / txt_df.shape[0])

"""
# LEARNING:
# 1. The first reason in the 'Discharge Diagnosis' section could be a proxy as a 'validation' value for the primary diagnosis, to be used for testing
"""

0.3432343234323432


"\n# LEARNING:\n# 1. The first reason in the 'Discharge Diagnosis' section could be a proxy as a 'validation' value for the primary diagnosis, to be used for testing\n"

In [11]:
"""
# QUESTION: How many drugs in each document do not have a 'Reason' entity?
"""
import statistics


rel_df_reason_drug = rel_df[rel_df['category'] == 'Reason-Drug']
# Filter ent_df to only have 'Drug' category
ent_df_drug = ent_df[ent_df['category'] == 'Drug']

averages = []
for file_idx in ent_df_drug['file_idx'].unique().tolist():
    drugs_in_rel_df = rel_df_reason_drug[rel_df_reason_drug['file_idx'] == file_idx]['entity2_text'].unique().tolist()
    drugs_in_ent_df = ent_df_drug[ent_df_drug['file_idx'] == file_idx]['text'].unique().tolist()
    drugs_without_reason = [drug for drug in drugs_in_ent_df if drug not in drugs_in_rel_df]
    averages.append(len(drugs_without_reason) / len(drugs_in_ent_df))
print(sum(averages) / len(averages))
print(statistics.median(averages))

"""
# LEARNING:
# 1. ~65% of drugs in each document do not have an 'Reason' entity when evaluating the 'Reason-Drug' relationship in rel_df
"""

0.647780516529275
0.6410256410256411


"\n# LEARNING:\n# 1. ~65% of drugs in each document do not have an 'Reason' entity when evaluating the 'Reason-Drug' relationship in rel_df\n"

In [12]:
"""
# QUESTION: How many docuements have a 'Discharge Diagnosis' section?
"""
# Loop through each document in txt_df and check if 'Discharge Diagnosis' exists in the 'text' column
discharge_diagnosis = 0
for index, row in txt_df.iterrows():
    if 'discharge diagnosis' in row['text'].lower():
        discharge_diagnosis += 1
print(discharge_diagnosis / txt_df.shape[0])
"""
# LEARNING:
# 1. 91% of docuements have a 'Discharge Diagnosis' section. We can reliably use this section to find the primary diagnosis.
"""

0.9141914191419142


"\n# LEARNING:\n# 1. 91% of docuements have a 'Discharge Diagnosis' section. We can reliably use this section to find the primary diagnosis.\n"

In [13]:
"""
# QUESTION: How many entities in ent_df fall in the 'Discharge Diagnosis' section?
"""
print(ent_df.head())
# Loop through each entity in ent_df and check if it is in the 'Discharge Diagnosis' section
discharge_diagnosis_entities = 0
for index, row in ent_df.iterrows():
    if row['section_Discharge Diagnosis'] == 1:
        discharge_diagnosis_entities += 1
print(discharge_diagnosis / ent_df.shape[0])

"""
# LEARNING:
# 1. ~1% of entities in ent_df fall in the 'Discharge Diagnosis' section for their respective docuement. We cannot reliably use entities from ent_df to find the primary medical diagnosis from this section of text.
"""

  file_idx entity_id  category  start_idx  end_idx           text  \
0   120253        T1      Drug      10002    10015  calcipotriene   
1   120253        T2  Strength      10016    10021          0.005   
2   120253        T3      Form      10024    10029          cream   
3   120253        T4    Dosage      10035    10042      one ( 1 )   
4   120253        T5      Form      10043    10047           appl   

        orig_txt  count_in_document  section_Chief Complaint  \
0  calcipotriene                  1                        0   
1          0.005                  2                        0   
2          cream                  2                        0   
3        one (1)                  5                        0   
4           appl                  1                        0   

   section_Discharge Diagnosis  section_History of Present Illness  \
0                            0                                   0   
1                            0                              

"\n# LEARNING:\n# 1. ~1% of entities in ent_df fall in the 'Discharge Diagnosis' section for their respective docuement. We cannot reliably use entities from ent_df to find the primary medical diagnosis from this section of text.\n"